In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display
%matplotlib inline

In [2]:
# Read data and drop redundant column.
data = pd.read_csv('epl_train.csv')

# Preview data.
data = data[data.MW > 3]

data.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

cleanup_nums = {"FTR":     {"H": 0, "D": 1, "A": 2}}
data = data.replace(cleanup_nums)
display(data.head())

,FTR,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,0,1.00,1.50,D,L,W,W,L,W,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,0,2.25,1.00,W,W,W,D,W,L,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,1,0.75,0.75,L,W,L,W,L,L,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,0,1.50,0.75,W,W,L,L,L,W,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0,0.75,0.75,W,L,L,L,L,W,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [3]:
X_all = data.drop(['FTR'],1)
y_all = data['FTR']

cols = [['HTGD','ATGD','HTP','ATP','DiffLP']]

In [4]:
#last 3 wins
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

def features(X):
    
    temp = pd.DataFrame(index = X.index)

    # Check each column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised version
        temp = temp.join(col_data)
    
    return temp

X_all = features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [5]:
print("Feature values:")
display(X_all.head())

Feature values:


,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.00,1.50,1,0,0,0,1,0,0,0,...,0,0,1,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,2.25,1.00,0,0,1,0,0,1,0,0,...,0,1,0,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,0.75,0.75,0,1,0,0,0,1,0,1,...,0,1,0,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,1.50,0.75,0,0,1,0,0,1,0,1,...,0,0,1,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0.75,0.75,0,0,1,0,1,0,0,1,...,0,0,1,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [6]:
from sklearn.model_selection import train_test_split

# Random split dataset into training and testing set 4:1 ratio.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 2,
                                                    stratify = y_all)

In [7]:
from time import time 
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, pos_label='H',average='micro'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [12]:
# Initialize XGboost
clf_xgb = xgb.XGBClassifier(seed = 79)
train_predict(clf_xgb, X_train, y_train, X_test, y_test)

Training a XGBClassifier using a training set size of 3079. . .
[16:03:11] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Trained model in 1.6888 seconds
Made predictions in 0.0170 seconds.
0.9983760961351088 0.9983760961351088
F1 score and accuracy score for training set: 0.9984 , 0.9984.
Made predictions in 0.0140 seconds.
F1 score and accuracy score for test set: 0.5532 , 0.5532.


In [14]:
# Hyper paramter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report, confusion_matrix
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='H')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid = grid.fit(X_train,y_train)

# Get the estimator
print("Best Parameters: ",grid.best_params_)
print("Best Estimators: ",grid.best_estimator_)
print(classification_report(y_test, grid.predict(X_test)))

[16:03:24] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:03:25] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the ol

In [17]:
# Metric results after tuning
clf_xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, monotone_constraints='()',
              n_estimators=40, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=2,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=None, seed=2,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)
train_predict(clf_xgb, X_train, y_train, X_test, y_test)

Training a XGBClassifier using a training set size of 3079. . .
[16:11:13] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Trained model in 0.4827 seconds
Made predictions in 0.0080 seconds.
0.5924001299123092 0.5924001299123092
F1 score and accuracy score for training set: 0.5924 , 0.5924.
Made predictions in 0.0060 seconds.
F1 score and accuracy score for test set: 0.5675 , 0.5675.


In [18]:
#Importing test set
from sklearn.metrics import classification_report, confusion_matrix
epl = pd.read_csv('epl_test.csv')

epl = epl[epl.MW > 3]

epl.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)
cleanup_nums = {"FTR":     {"H": 0, "D": 1, "A": 2}}
epl = epl.replace(cleanup_nums)
X1_all = epl.drop(['FTR'],1)
y1_all = epl['FTR']
X1_all.HM1 = X1_all.HM1.astype('str')
X1_all.HM2 = X1_all.HM2.astype('str')
X1_all.HM3 = X1_all.HM3.astype('str')
X1_all.AM1 = X1_all.AM1.astype('str')
X1_all.AM2 = X1_all.AM2.astype('str')
X1_all.AM3 = X1_all.AM3.astype('str')

#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X1_all = preprocess_features(X1_all)
print("Processed feature columns ({} total features):\n{}".format(len(X1_all.columns), list(X1_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [20]:
# Checking metrics using test set
clf_xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, monotone_constraints='()',
              n_estimators=40, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=2,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=None, seed=2,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)
clf_xgb.fit(X1_all,y1_all)
f1, acc = predict_labels(clf_xgb, X1_all, y1_all)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
print(classification_report(y1_all, clf_xgb.predict(X1_all)))

[16:25:20] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Made predictions in 0.0070 seconds.
F1 score and accuracy score for training set: 0.7800 , 0.7800.
              precision    recall  f1-score   support

           0       0.75      0.85      0.80       133
           1       0.93      0.53      0.68        81
           2       0.76      0.86      0.81       136

    accuracy                           0.78       350
   macro avg       0.82      0.75      0.76       350
weighted avg       0.80      0.78      0.77       350



In [22]:
#Predictions
y1_predict = clf_xgb.predict(X1_all)
print(y1_predict)

[1 0 2 0 0 0 2 2 2 0 0 2 0 1 0 2 0 0 2 2 2 1 0 0 0 2 1 2 0 2 2 0 2 0 2 0 0
 2 1 0 2 2 0 0 2 0 1 2 2 0 0 0 2 2 1 0 0 2 0 1 0 2 0 2 1 2 0 0 2 0 2 0 2 2
 0 0 2 2 2 0 1 0 0 2 2 2 0 2 0 2 0 2 1 0 0 0 2 1 2 0 1 0 2 0 2 0 0 1 0 1 2
 0 2 0 1 0 2 1 1 2 0 1 2 0 1 2 0 0 0 1 2 2 2 2 2 2 2 0 0 0 0 1 0 2 0 2 1 0
 0 0 2 0 2 0 0 2 2 2 2 2 0 1 1 0 2 0 0 0 1 1 2 0 2 2 2 0 2 0 2 2 2 2 2 0 2
 2 1 0 2 0 0 1 2 2 0 0 2 0 1 2 2 0 0 0 0 1 2 2 2 0 2 0 0 2 2 0 0 0 0 0 2 1
 1 2 0 1 2 0 0 1 2 1 2 2 2 2 2 1 2 1 2 0 0 0 0 0 1 1 0 2 2 2 0 0 0 2 2 0 2
 2 0 0 2 2 0 2 0 0 1 2 2 0 0 2 2 0 2 2 0 2 2 2 0 0 0 2 0 0 2 0 2 0 2 0 2 1
 0 0 1 2 0 0 2 2 0 1 2 0 1 2 2 0 0 2 2 0 0 2 0 2 1 2 2 2 0 2 2 0 2 0 0 2 2
 2 0 0 2 2 2 2 0 2 2 0 2 0 0 2 0 2]


In [23]:
# Checking returns using Logistic Regression
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.00

for i in range(len(X1_all)):
    addition = 30+i
    prediction = y1_predict[i]
    print('\nMatch',i+1)
    print('\nPrediction', prediction)
    print('Actual', y1_all.iloc[i])
    print('Favourite', np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]))
    print('Home, Draw and Away odds', epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A'])      
          
    
    if prediction == 2:
            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]): 
                                
                favourites +=1
                
            if  prediction == y1_all.iloc[i]:
                funds += (wager * epl.loc[addition,'B365A']) - wager
            else:
                funds -= wager
            
    elif prediction == 1:
            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]):  

                favourites +=1
            if  prediction == y1_all.iloc[i]:
                funds +=( wager * epl.loc[addition,'B365D']) - wager
            else:
                funds -= wager

    else:

            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]):
                favourites +=1
            if  prediction == y1_all.iloc[i]:
                funds += (wager * epl.loc[addition,'B365A']) - wager
            else:
                funds -= wager
   
    print('Funds', funds)
    

print(f'No bet placed {no_bets} times')


Match 1

Prediction 1
Actual 1
Favourite 2
Home, Draw and Away odds 7.5 5.25 1.36
Funds 142.5

Match 2

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 2.4 3.2 3.1
Funds 163.5

Match 3

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 1.61 4.2 5.25
Funds 206.0

Match 4

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.72 3.8 4.75
Funds 243.5

Match 5

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.57 4.0 6.0
Funds 293.5

Match 6

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.5 4.0 7.5
Funds 358.5

Match 7

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 1.85 3.8 4.0
Funds 388.5

Match 8

Prediction 2
Actual 0
Favourite 2
Home, Draw and Away odds 10.0 5.5 1.3
Funds 378.5

Match 9

Prediction 2
Actual 1
Favourite 2
Home, Draw and Away odds 3.75 4.2 1.83
Funds 368.5

Match 10

Prediction 0
Actual 1
Favourite 0
Home, Draw and Away odds 1.45 4.75 6.5
Funds 358.5

Match 11

Prediction 0
Actual 0
Favourite 0
Home, Draw 

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.72 4.2 4.2
Funds 3002.0000000000005

Match 128

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.75 4.1 4.2
Funds 3034.0000000000005

Match 129

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.95 3.5 3.9
Funds 3063.0000000000005

Match 130

Prediction 1
Actual 1
Favourite 0
Home, Draw and Away odds 2.62 3.25 2.75
Funds 3085.5000000000005

Match 131

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 5.75 4.0 1.57
Funds 3091.2000000000003

Match 132

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 5.25 4.2 1.61
Funds 3097.3

Match 133

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 3.0 3.7 2.2
Funds 3109.3

Match 134

Prediction 2
Actual 0
Favourite 2
Home, Draw and Away odds 5.0 4.33 1.61
Funds 3099.3

Match 135

Prediction 2
Actual 0
Favourite 0
Home, Draw and Away odds 2.45 3.1 3.1
Funds 3089.3

Match 136

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 7.0 


Match 279

Prediction 0
Actual 0
Favourite 2
Home, Draw and Away odds 3.6 3.3 2.1
Funds 6596.300000000001

Match 280

Prediction 2
Actual 1
Favourite 0
Home, Draw and Away odds 2.25 3.3 3.3
Funds 6586.300000000001

Match 281

Prediction 2
Actual 1
Favourite 2
Home, Draw and Away odds 3.4 3.5 2.1
Funds 6576.300000000001

Match 282

Prediction 2
Actual 0
Favourite 2
Home, Draw and Away odds 3.5 3.5 2.05
Funds 6566.300000000001

Match 283

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.66 3.5 5.75
Funds 6613.800000000001

Match 284

Prediction 0
Actual 1
Favourite 0
Home, Draw and Away odds 1.7 3.8 4.8
Funds 6603.800000000001

Match 285

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.33 5.25 9.0
Funds 6683.800000000001

Match 286

Prediction 2
Actual 1
Favourite 2
Home, Draw and Away odds 4.5 4.33 1.66
Funds 6673.800000000001

Match 287

Prediction 0
Actual 1
Favourite 0
Home, Draw and Away odds 1.57 4.0 6.0
Funds 6663.800000000001

Match 288

Prediction 0
Act